# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
#nipa_series_codes(retrieve_table('T20100'))

### GDP components

In [3]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

srs = {'A191RX': '\hspace{0.5mm} {\color{red!95!black}\\textbf{---}} Gross Domestic Product', 
       'DPCERX': '\hspace{2.5mm} {\color{yellow!65!orange}\\textbf{---}} Consumer Spending', 
       'A006RX': '\hspace{2.5mm} {\color{blue!70!black}\\textbf{---}} Gross Private Domestic Investment', 
       'A822RX': '\hspace{2.5mm} {\color{cyan!60!white}\\textbf{---}} Government Spending and Investment',
       'A019RX': '\hspace{2.5mm} {\color{green!60!black}\\textbf{---}} Net Exports', 
       'A020RX': '\hspace{7.5mm} Exports', 'A021RX': '\hspace{7.5mm} Less: Imports'}
s = [key for key, value in srs.items()]
s2 = [i[:-1].replace('A02', 'B02') + 'C' for i in s]
df = nipa_df(retrieve_table('T10106')['Data'], s).sort_index()
df2 = nipa_df(retrieve_table('T10105')['Data'], s2).sort_index()

dfpop = df.div(pop, axis=0)
real_vals = df2.rename({i: i.replace('RC', 'RX').replace('B02', 'A02') 
                        for i in s2}, axis=1).iloc[-1]
data = ((dfpop / dfpop.iloc[-1]) * (real_vals / pop.iloc[-1])).loc['1989':]

sel_cols = ['DPCERX', 'A006RX', 'A019RX', 'A822RX']

data[sel_cols].to_csv(data_dir / 'gdp_pc_comp.csv', index_label='date')

In [4]:
pce = f'\${data.DPCERX.iloc[-1] * 1000:,.0f}'
inv = f'\${data.A006RX.iloc[-1] * 1000:,.0f}'
gov = f'\${data.A822RX.iloc[-1] * 1000:,.0f}'
exp = f'\${abs(data.A019RX.iloc[-1]) * 1000:,.0f}'
ltdt = dtxt(data.index[-1])['qtr1']
pce_diff = f'\${(data.DPCERX.iloc[-1] - data.DPCERX.iloc[0]) * 1000:,.0f}'

text = ('Much of the increase in real GDP per capita over the past '+
        '30 years comes from consumer spending. Domestic '+
        'consumer spending (see {\color{yellow!65!orange}\\textbf{---}}) '+
        f'is equivalent to {pce} per person in {ltdt}, a price-adjusted increase '+
        f'of {pce_diff} since 1989. Gross private domestic investment '+
        '(see {\color{blue!70!black}\\textbf{---}}) is equivalent to '+
        f'{inv} per person in {ltdt}, and government spending and '+
        'investment (see {\color{cyan!60!white}\\textbf{---}}) totals '+
        f'{gov} per person. Net exports equivalent to {exp} per person '+
        'are subtracted to reflect only domestic production '+
        '(see {\color{green!60!black}\\textbf{---}}).')

write_txt(text_dir / 'gdp_pc_comp.txt', text)
write_txt(text_dir / 'gdp_ltdt.txt', ltdt)
print(text)

Much of the increase in real GDP per capita over the past 30 years comes from consumer spending. Domestic consumer spending (see {\color{yellow!65!orange}\textbf{---}}) is equivalent to \$43,957 per person in 2020 Q4, a price-adjusted increase of \$17,458 since 1989. Gross private domestic investment (see {\color{blue!70!black}\textbf{---}}) is equivalent to \$11,840 per person in 2020 Q4, and government spending and investment (see {\color{cyan!60!white}\textbf{---}}) totals \$11,584 per person. Net exports equivalent to \$2,431 per person are subtracted to reflect only domestic production (see {\color{green!60!black}\textbf{---}}).


In [5]:
res = data * 1000
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).loc['2019-10-01']
pr.name = dtxt(pr.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).iloc[0]
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdppc_levels.tex', sep='&', line_terminator='\\\ ', quotechar=' ')
table

,2020 Q4,2019 Q4,2000 Q1,1989 Q1
\hspace{0.5mm} {\color{red!95!black}\textbf{---}} Gross Domestic Product,"\$64,950","66,909","52,558","42,245"
\hspace{2.5mm} {\color{yellow!65!orange}\textbf{---}} Consumer Spending,"43,957","45,378","33,883","26,498"
\hspace{2.5mm} {\color{blue!70!black}\textbf{---}} Gross Private Domestic Investment,"11,840","11,502","8,900","5,728"
\hspace{2.5mm} {\color{cyan!60!white}\textbf{---}} Government Spending and Investment,"11,584","11,711","10,862","10,609"
\hspace{2.5mm} {\color{green!60!black}\textbf{---}} Net Exports,"-2,431","-1,874","-1,315",-438
\hspace{7.5mm} Exports,"6,674","7,537","4,611","2,404"
\hspace{7.5mm} Less: Imports,"9,105","9,205","5,840","2,739"


### Labor Share

In [6]:
s = ['A261RC', 'A4002C']

df = nipa_df(retrieve_table('T11000')['Data'], s).sort_index()
df['Share'] = (df['A4002C'] / df['A261RC']) * 100
data = df.loc['1989':, 'Share'].dropna()
data.to_csv(data_dir / 'laborshare.csv', 
            index_label='date', 
            float_format='%g')

node = end_node(data, 'blue!60!cyan')
write_txt(text_dir / 'laborshare_node.txt', node)

In [7]:
s = series_info(data)
ltdate = dtxt(s['date_latest'])['qtr2']
one_yr = val_inc_pp(s['val_latest'] - s['val_year_ago'])
ltmin = s['val_latest'] - s['val_min']
ltmax = s['val_max'] - s['val_latest']
text = (f'As of {ltdate}, labor receives '+
        f'{s["val_latest"]:.1f} percent of gross domestic income, '+
        f'and the labor share {one_yr} over the past year. '+
        f'The labor share is {ltmin:.1f} percentage '+
        f'points above its 30-year low of {s["val_min"]:.1f} percent '+
        f'in {s["date_min_ft"]}, but {ltmax:.1f} percentage points '+
        f'below the 30-year high of {s["val_max"]:.1f} percent in '+
        f'{s["date_max_ft"]}. ')

write_txt(text_dir / 'laborshare.txt', text)
text

'As of the third quarter of 2020, labor receives 55.2 percent of gross domestic income, and the labor share increased by a total of 1.8 percentage points over the past year. The labor share is 3.6 percentage points above its 30-year low of 51.6 percent in 2014 Q3, but 2.3 percentage points below the 30-year high of 57.4 percent in 1992 Q3. '

### GDP growth rate

In [8]:
s = ['A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
df.loc['1989':].to_csv(data_dir / 'gdp.csv', index_label='date')
date = dtxt(df.index[-1])['qtr1']

txt = f'{date}: {df["A191RL"].iloc[-1]}\%'
write_txt(data_dir / 'gdp.txt', txt)

In [9]:
df = nipa_df(retrieve_table('T10502')['Data'], ['A191RL'])['A191RL']
ltdate = dtxt(df.index[-1])['qtr2']
prdate = dtxt(df.index[-2])['qtr2']
prdate2 = dtxt(df.index[-3])['qtr1']
pravg = df['2017': '2019'].values.mean().round(1)
d = {}
for i in [-13, -41, -121]:
    d[i] = df.iloc[i:].values.mean().round(1)
    
text = (f'{inc_dec_percent(df.iloc[-1], annualized=True)} during {ltdate}, '+
        f'compared to {inc_dec_percent(df.iloc[-2], how="of")} in {prdate}, and '+
        f'{inc_dec_percent(df.iloc[-3], how="of")} in {prdate2}. This '+
        f'annualized rate of quarterly growth averaged {pravg} percent over '+
        f'the three years from 2017--2019. Over the most recent three years, '+
        'which include the COVID-19 pandemic, real GDP '+
        f'growth averaged {d[-13]} percent.')    
write_txt(text_dir / 'gdp_gr.txt', text)
print(text)

increased at an annual rate of 4.1 percent during the fourth quarter of 2020, compared to an increase of 33.4 percent in the third quarter of 2020, and a decrease of 31.4 percent in 2020 Q2. This annualized rate of quarterly growth averaged 2.5 percent over the three years from 2017--2019. Over the most recent three years, which include the COVID-19 pandemic, real GDP growth averaged 1.9 percent.


### GDP Intro Chart and Text

In [10]:
df = nipa_df(retrieve_table('T70100')['Data'], ['A939RC', 'A939RX'])
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1]) * df['A939RC'].iloc[-1]
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
rgdp_cd = rgdp / rgdp.iloc[-1] * cd

ltdate = dtxt(df.index[-1])['qtr2']
gdp_val = int(rgdp_cd.iloc[-1].values[0] / 1000)
gdp_prv_val = int(rgdp_cd.loc['2019-10-01'] / 1000)
gdp_prv_val2 = int(rgdp_cd.iloc[0].values[0] / 1000)
gdppc_val = int(df.iloc[-1].loc['value'])
gdppc_prv_val = int(df.loc['2019-10-01', 'value'])
gdppc_prv_val2 = int(df.iloc[0].loc['value'])
qdate = dtxt(df.index[-1])['qtr1']

text = (f'\${gdp_val:,} billion in {ltdate}, compared to an '+
        f'inflation-adjusted equivalent of \${gdp_prv_val:,} '+
        f'billion in 2019 Q4 and \${gdp_prv_val2:,} billion in '+
        'the first quarter of 1989.')
write_txt(text_dir / 'gdp1.txt', text)
print(text, '\n\n')

text = ('The US population is growing by about sixth-tenths of a '+
        'percent per year. GDP per capita (see {\color{red!95!black}\\textbf{---}} ), '+
        f'adjusted for inflation to {qdate} dollars, had increased to '+
        f'\${gdppc_prv_val:,} in 2019 Q4 from \${gdppc_prv_val2:,} in 1989 Q1, and is '+
        f'currently \${gdppc_val:,}.')
write_txt(text_dir / 'gdp2.txt', text)
print(text, '\n\n')

date = df.index[-1].strftime('%Y-%m-%d')
ldate = dtxt(df.index[-1])['qtr4']

text = [r'\noindent \normalsize \textbf{GDP per capita}\\',
r'\footnotesize{\textit{in ' + qdate + r' dollars}}\\* ',
r'\noindent \begin{tikzpicture}',
r'    \begin{axis}[\dateaxisticks clip=false, height=5.2cm, width=5.8cm,',
r'            ytick={40000,50000,60000,70000}, yticklabels={40, 50, 60, \$70k},',
r'            xtick={{1989-01-01}, {2000-01-01}, {2010-01-01}, {2019-04-01}},',
r'        minor xtick={}, ',
f"        xticklabels={{`89, `00, `10, {ldate}}}, enlarge y limits={{lower, 0.3}}, ",
r'        enlarge x limits={0.05}]',
r'    \stdline{red!95!black}{date}{value}{data/gdppc.csv}',
r'    \node[label={90:{\$'+ f'{gdppc_val:,}' + r'}}, circle, red!95!black, fill, inner sep=1.5pt] at ',
f'        (axis cs:{date},{gdppc_val})' + r' {};',
r'    \end{axis}',
r'\end{tikzpicture}\\',
r'\footnotesize{Source: Bureau of Economic Analysis}']

f = text_dir.joinpath('gdp_chart.txt')
with f.open('w') as wf:
    for t in text:
        print(t)
        wf.write(t + '\n')

\$21,487 billion in the fourth quarter of 2020, compared to an inflation-adjusted equivalent of \$22,025 billion in 2019 Q4 and \$9,980 billion in the first quarter of 1989. 


The US population is growing by about sixth-tenths of a percent per year. GDP per capita (see {\color{red!95!black}\textbf{---}} ), adjusted for inflation to 2020 Q4 dollars, had increased to \$66,910 in 2019 Q4 from \$40,871 in 1989 Q1, and is currently \$64,951. 


\noindent \normalsize \textbf{GDP per capita}\\
\footnotesize{\textit{in 2020 Q4 dollars}}\\* 
\noindent \begin{tikzpicture}
    \begin{axis}[\dateaxisticks clip=false, height=5.2cm, width=5.8cm,
            ytick={40000,50000,60000,70000}, yticklabels={40, 50, 60, \$70k},
            xtick={{1989-01-01}, {2000-01-01}, {2010-01-01}, {2019-04-01}},
        minor xtick={}, 
        xticklabels={`89, `00, `10, `20 Q4}, enlarge y limits={lower, 0.3}, 
        enlarge x limits={0.05}]
    \stdline{red!95!black}{date}{value}{data/gdppc.csv}
    \node[labe

### Private fixed investment

In [11]:
# Private fixed investment contribution to growth
s = ['A008RY', 'A011RY', 'A014RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)

df.loc['1989':].to_csv(data_dir / 'inv.csv', index_label='date')

ldate = df.index[-1]
datetext = dtxt(ldate)["qtr1"]

tot_contr = cont_subt(df.loc[ldate, ['A008RY', 'A011RY']].sum())
bus_contr = cont_subt(df.loc[ldate, 'A008RY'], 'end')
res_contr = cont_subt(df.loc[ldate, 'A011RY'], 'end')
inv_contr = cont_subt(df.loc[ldate, 'A014RY'], 'end')

text = (f'During the quarter, private fixed investment {tot_contr} real GDP growth. '+
        f'Non-residential fixed investment {bus_contr}, while '+
        f'residential fixed investment {res_contr}. The change in private '+
        f'inventories {inv_contr}.')

write_txt(text_dir / 'inv_text2.txt', text)

In [12]:
# In levels and share of GDP
s = ['A191RC', 'A007RC', 'A008RC', 'A011RC']

df = nipa_df(retrieve_table('T10105')['Data'], s)

ldate = df.index[-1]

inv_tot = df.loc[ldate, 'A007RC'] / 1_000_000
gdp_tot = df.loc[ldate, 'A191RC'] / 1_000_000
res_tot = df.loc[ldate, 'A011RC'] / 1_000
bus_tot = df.loc[ldate, 'A008RC'] / 1_000_000

inv_sh = (inv_tot / gdp_tot) * 100
res_sh = (res_tot / 1_000 / gdp_tot) * 100
bus_sh = (bus_tot / gdp_tot) * 100

text = (f'In {dtxt(ldate)["qtr2"]}, private fixed investment, '+
        f'which does not include inventory investment, totals \${inv_tot:.1f} trillion, '+
        f'equivalent to {inv_sh:.1f} percent of GDP. Non-residential (business) fixed '+
        f'investment totals \${bus_tot:.1f} trillion, or {bus_sh:.1f} percent of GDP, '+
        f'while residential fixed investment totals \${res_tot:.1f} billion ({res_sh:.1f} '+
        'percent of GDP).')

write_txt(text_dir / 'inv_text.txt', text)

### Imports and Exports

In [13]:
# Import and Export share of GDP
s = ['B020RC', 'B021RC', 'B648RC', 'LA000006']
s2 = ['A191RC']
df = nipa_df(retrieve_table('T40205')['Data'], s)
df['A191RC'] = nipa_df(retrieve_table('T10105')['Data'], s2)
df['EX'] = df['B020RC'] - df['LA000006']
df['IM'] = df['B021RC'] - df['B648RC']
data = df.div(df['A191RC'], axis=0) * 100
data.loc['1989':].to_csv(data_dir / 'eximgdp.csv', index_label='date')

date = f'{qtrs[data.index[-1].quarter]} quarter of {data.index[-1].year}'
valex = data['EX'].iloc[-1]
valim = data['IM'].iloc[-1]

text = (f'Nonpetroleum goods and services imports (see {{\color{{green!60!teal!80!black}}'+
        f'\\textbf{{---}}}}) were equivalent to {valim:.1f} percent of GDP in the {date}, '+
        f'while exports of nonpetroleum goods and services (see {{\\color{{blue!90!cyan}}'+
        f'\\textbf{{---}}}}) were equivalent to {valex:.1f} percent of GDP.')

write_txt(text_dir / 'exim.txt', text)

print(text)

Nonpetroleum goods and services imports (see {\color{green!60!teal!80!black}\textbf{---}}) were equivalent to 13.4 percent of GDP in the fourth quarter of 2020, while exports of nonpetroleum goods and services (see {\color{blue!90!cyan}\textbf{---}}) were equivalent to 9.6 percent of GDP.


### Goods Import Penetration

In [14]:
s = ['A353RC']

G = nipa_df(retrieve_table('T10205')['Data'], s).sort_index()

s = ['A253RC', 'A255RC', 'B647RC', 'LA000004', 'A650RC', 'B651RC', 'A652RC', 'A653RC', 'B648RC']

MX = nipa_df(retrieve_table('T40205')['Data'], s).sort_index()

D = G['A353RC'] - MX['A253RC'] + MX['A255RC']
result = (MX['A255RC'] / D)

import_categories = ['B647RC', 'LA000004', 'A650RC', 'B651RC', 
                     'A652RC', 'A653RC', 'B648RC']
Msh = MX[import_categories].div(MX['A255RC'], axis=0)

Msh['Consumer'] = Msh['B647RC'] + Msh['A652RC'] + Msh['B651RC']
Msh['Capital'] = Msh['LA000004'] - Msh['B648RC'] + Msh['A650RC'] + Msh['A653RC']

final = Msh[['Consumer', 'Capital', 'B648RC']].multiply(result, axis=0) * 100

final.loc['1989':].to_csv(data_dir / 'goodsimpsh.csv', index_label='date')

ltdate = final.index[-1]
datetext = dtxt(ltdate)['qtr1']
cons = final['Consumer'].iloc[-1]
capi = final['Capital'].iloc[-1]
oil = final['B648RC'].iloc[-1]

text = (f'As of {datetext}, imports of consumer goods excluding petroleum '+
        f'and petroleum products are equivalent to {cons:.1f} percent of domestic '+
        'consumption of goods (see\cbox{cyan!40!white}). Petroleum-related imports '+
        f'claim {oil:.1f} percent (see\cbox{{purple}}) and imports of all other goods, '+
        f'primarily capital goods, industrial supplies, and materials are equivalent to '+
        f'{capi:.1f} percent (see\cbox{{blue!50!cyan}}).')

write_txt(text_dir / 'goodsimpsh1.txt', text)

ch11 = (final.loc['2011-01-01'] - final.iloc[0])

chlt = (final.iloc[-1] - final.loc['2011-01-01'])

chtxt = {}
for i, v in chlt.items():
    if v >= 0.1:
        chtxt[i] = f'increased by the equivalent of {v:.1f} percent'
    elif v <= -0.1:
        chtxt[i] = f'decreased by the equivalent of {abs(v):.1f} percent'
    else:
        chtxt[i] = 'was virtually unchanged'
        
text = ('From 1989 to 2011, imports of consumer goods excluding petroleum increased '+
        f'by the equivalent of {ch11["Consumer"]:.1f} percent of domestic '+
         'consumption of goods; petroleum and products imports '+
        f'increased by the equivalent of {ch11["B648RC"]:.1f} percent; '+
        'and all other goods increased by the equivalent '+
        f'of {ch11["Capital"]:.1f} percent. Since 2011, imports of '+
        f'consumer goods {chtxt["Consumer"]} of domestic goods demand; '+
        f'imports of petroleum and products {chtxt["B648RC"]}; and other '+
        f'imports {chtxt["Capital"]}.')

write_txt(text_dir / 'goodsimpsh.txt', text)

### ITA - Financial Account Balance

In [15]:
ind_list = ['FinAssetsExclFinDeriv', 'FinLiabsExclFinDeriv', 'FinDeriv', 'StatDisc']

api_results = bea_api_ita(ind_list, bea_key)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

results = pd.DataFrame({name: {i['TimePeriod']: i['DataValue'] 
                               for i in json.loads(series)['BEAAPI']['Results']['Data']} 
                        for name, series in api_results})
results.index = pd.to_datetime(results.index)
results = results.replace(r'^\s*$', np.nan, regex=True).astype('float').rolling(4).sum()
results['FAB'] = results['FinLiabsExclFinDeriv'] - results['FinAssetsExclFinDeriv']
results['TOT'] = results[['FAB', 'StatDisc', 'FinDeriv']].sum(axis=1)
final = (results.divide(gdp, axis=0).dropna(how='all') * 100).fillna(0)
final[['FinDeriv', 'StatDisc', 'FAB', 'TOT']].to_csv(data_dir/'fab.csv', index_label='date')

s = final.iloc[-1]
liab = s.FinLiabsExclFinDeriv
assets = s.FinAssetsExclFinDeriv

ldate = dtxt(final.index[-1])['qtr1']

text = (f'Over the year ending {ldate}, net domestic acquisitions of foreign assets were '+
        f'equivalent to {assets:.1f} percent of GDP, while net domestic incurrence of foreign '+
        f'liabilities total {liab:.1f} percent of GDP. Domestic net borrowing totals '+
        f'{s.TOT:.1f} percent of GDP.')
write_txt(text_dir / 'fab.txt', text)
print(text)

Over the year ending 2020 Q3, net domestic acquisitions of foreign assets were equivalent to 2.6 percent of GDP, while net domestic incurrence of foreign liabilities total 5.4 percent of GDP. Domestic net borrowing totals 2.6 percent of GDP.


### GDP Composition

In [16]:
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY']

df = nipa_df(retrieve_table('T10502')['Data'], s).loc['1989':]
df.to_csv(data_dir / 'comp.csv', index_label='date')

ltdt = dtxt(df.index[-1])['qtr2']

sl = [('DPCERY', 'main'), ('A006RY', 'main'), 
      ('A822RY', 'end'), ('A019RY', 'end')]

d = {}

for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)
    
text = (f'In {ltdt}, consumer spending (see\cbox{{yellow!80!orange}}) '+
        f'{d["DPCERY"]} overall real GDP growth. Private domestic '+
        f'investment (see\cbox{{blue!70!black}}) {d["A006RY"]} real GDP '+
        'growth, government spending and investment (see\cbox{cyan!50!white}) '+
        f'{d["A822RY"]}, and net exports (see\cbox{{green!60!black}}) '+
        f'{d["A019RY"]}.')  

write_txt(text_dir / 'gdp_exp_basic.txt', text)
print(text)

In the fourth quarter of 2020, consumer spending (see\cbox{yellow!80!orange}) contributed 1.61 percentage points to overall real GDP growth. Private domestic investment (see\cbox{blue!70!black}) contributed 4.23 percentage points to real GDP growth, government spending and investment (see\cbox{cyan!50!white}) subtracted 0.19 percentage point, and net exports (see\cbox{green!60!black}) subtracted 1.55 percentage points.


### GDP per capita

In [17]:
s = ['A939RC', 'A939RX']

df = nipa_df(retrieve_table('T70100')['Data'], s)
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1])  * df['A939RC'].iloc[-1]
df[['value']].loc['1989':].to_csv(data_dir / 'gdppc.csv', index_label='date')

### Gross Domestic Income

In [14]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)

cols = ['A261RC', 'W271RC', 'A4002C', 'A262RC', 'indirect']

contr.loc['1989':, cols].to_csv(data_dir / 'gdi.csv', index_label='date')

dfpop = df.div(pop, axis=0).dropna()

ltdt = dtxt(dfpop.index[-1])['qtr1']
write_txt(text_dir / 'gdi_ltdt.txt', ltdt)

dfpop[cols].to_csv(data_dir / 'gdi_pc_comp.csv', index_label='date')

In [13]:
nipa_df(retrieve_table('T11000')['Data'], s)

,A261RC,A4002C,W056RC,A107RC,W271RC,A262RC,A4102C,A038RC
1988-01-01,5068055.0,2858473.0,365220.0,30364.0,1110925.0,763802.0,2367096.0,491377.0
1988-04-01,5176274.0,2924054.0,372500.0,29842.0,1131882.0,777680.0,2422462.0,501592.0
1988-07-01,5285530.0,2976558.0,377526.0,29237.0,1169312.0,791371.0,2463571.0,512987.0
1988-10-01,5403861.0,3036327.0,382612.0,28565.0,1208352.0,805134.0,2509467.0,526860.0
1989-01-01,5493644.0,3080126.0,391008.0,27952.0,1232034.0,818428.0,2541872.0,538254.0
...,...,...,...,...,...,...,...,...
2019-10-01,21721174.0,11577444.0,1509639.0,81147.0,5240792.0,3474446.0,9435088.0,2142356.0
2020-01-01,21671321.0,11686291.0,1530015.0,75093.0,5021128.0,3508979.0,9537956.0,2148335.0
2020-04-01,19533535.0,10957993.0,1395758.0,1086548.0,4731962.0,3534370.0,8917312.0,2040680.0
2020-07-01,20803079.0,11475760.0,1489241.0,1213480.0,5479246.0,3572311.0,9352896.0,2122864.0


In [15]:
srs = {'A261RC': 'Gross Domestic Income', 
       'A4002C': '\hspace{0.1mm} {\color{magenta!90!blue}\\textbf{---}} Labor', 
       'A4102C': '\hspace{6mm} Wages and Salaries',
       'A038RC': '\hspace{6mm} Supplements',
       'W271RC': '\hspace{0.1mm} {\color{yellow!60!orange}\\textbf{---}} Profit', 
       'indirect': '\hspace{0.1mm} {\color{violet}\\textbf{---}} Indirect Taxes', 
       'W056RC': '\hspace{6mm} Taxes on Production and Imports',
       'A107RC': '\hspace{6mm} Less: Subsidies',
       'A262RC': '\hspace{0.1mm} {\color{teal!60!white}\\textbf{---}} Depreciation'}

res = (dfpop[list(srs.keys())] * 1000).dropna()
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).loc['2019-10-01']
pr.name = dtxt(pr.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdipc_levels.tex', sep='&', line_terminator='\\\ ', quotechar=' ')
print(table)

prdate = '2019-10-01'
ltval = df['A261RC'].iloc[-1] / 1000
ltpc = dfpop['A261RC'].iloc[-1] * 1000
prval = df['A261RC'].loc[prdate] / 1000
prpc = dfpop['A261RC'].loc[prdate] * 1000
prdt = dtxt(pd.to_datetime(prdate))['qtr1']

text = ('The Bureau of Economic Analysis \href{https://www.bea.gov/data/gdp/gross-domestic-product}{report} '+
        'seasonally-adjusted and annualized '+
        '\href{https://www.bea.gov/help/glossary/gross-domestic-income-gdi}{Gross Domestic Income} '+
        f'(GDI) of \${ltval:,.0f} billion in {ltdt}, compared to an inflation-'+
        f'adjusted equivalent of \${prval:,.0f} billion in {prdt}. '+
        f'Real GDI per capita was \${ltpc:,.0f} in {ltdt} '+
        f'and \${prpc:,.0f} in {prdt}.')

write_txt(text_dir / 'gdi_levels.txt', text)
print(text)

l_pc = dfpop['A4002C'].iloc[-1] * 1000
l_pr = dfpop['A4002C'].loc[prdate] * 1000
k_pc = dfpop['W271RC'].iloc[-1] * 1000
k_pr = dfpop['W271RC'].loc[prdate] * 1000
t_pc = dfpop['indirect'].iloc[-1] * 1000
t_pr = dfpop['indirect'].loc[prdate] * 1000
d_pc = dfpop['A262RC'].iloc[-1] * 1000
d_pr = dfpop['A262RC'].loc[prdate] * 1000

text = ('Gross labor income per capita is equivalent '+
        f'to \${l_pc:,.0f} in {ltdt} '+
        '(see {\color{magenta!90!blue}\\textbf{---}}) '+
        f'and \${l_pr:,.0f} in 2019 Q4, '+
        'on an annualized, seasonally-adjusted, and inflation-'+
        f'adjusted basis. Profits per person total \${k_pc:,.0f} in {ltdt} '+
        '(see {\color{yellow!60!orange}\\textbf{---}}) '+
        f'and \${k_pr:,.0f} in {prdt}, following the same adjustments. '+
        f'Indirect taxes less subsidies per capita total \${t_pc:,.0f} in {ltdt} '+
        '(see {\color{violet}\\textbf{---}}) '+
        f'and \${t_pr:,.0f} in {prdt}. Lastly, depreciation per capita is \${d_pc:,.0f} in '+
        f'{ltdt} (see {{\color{{teal!60!white}}\\textbf{{---}}}}) and \${d_pr:,.0f} in {prdt}.')

write_txt(text_dir / 'gdi_levels_pc.txt', text)
print(text)

                                                     2020 Q3 2019 Q4 2000 Q1  \
Gross Domestic Income                               \$62,969  66,504  53,228   
\hspace{0.1mm} {\color{magenta!90!blue}\textbf{...    34,736  35,447  30,098   
\hspace{6mm} Wages and Salaries                       28,311  28,888  24,882   
\hspace{6mm} Supplements                               6,426   6,559   5,217   
\hspace{0.1mm} {\color{yellow!60!orange}\textbf...    16,585  16,046  11,980   
\hspace{0.1mm} {\color{violet}\textbf{---}} Ind...       835   4,374   3,417   
\hspace{6mm} Taxes on Production and Imports           4,508   4,622   3,653   
\hspace{6mm} Less: Subsidies                           3,673     248     236   
\hspace{0.1mm} {\color{teal!60!white}\textbf{--...    10,813  10,638   7,733   

                                                   1989 Q1  
Gross Domestic Income                               41,906  
\hspace{0.1mm} {\color{magenta!90!blue}\textbf{...  23,495  
\hspace{6mm} Wag

In [16]:
s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']

df = contr[s]
gdi_lt = inc_dec_percent(df["A261RC"].iloc[-1], annualized=True)
gdi_pr = inc_dec_percent(df["A261RC"].iloc[-2], how='of')
gdi_pr2 = inc_dec_percent(df["A261RC"].iloc[-3], how='of')

ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']
prdt2 = dtxt(df.index[-3])['qtr1']


sl = [('A4002C', 'main'), ('W271RC', 'main'), 
      ('indirect', 'end'), ('A262RC', 'end')]

d = {}
for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)
    
text1 = (f'In {ltdt}, gross domestic income {gdi_lt}, following {gdi_pr} '+
         f'in {prdt} and {gdi_pr2} in {prdt2}. ')  

l_lt = cont_subt(df['A4002C'].iloc[-1])
l_pr = cont_subt(df['A4002C'].iloc[-2], style='of')
k_lt = cont_subt(df['W271RC'].iloc[-1]).replace(' from', '').replace(' to', '')
k_pr = cont_subt(df['W271RC'].iloc[-2]).replace(' from', '').replace(' to', '')
if k_lt[:3] == k_pr[:3]:
    if abs(df['W271RC'].iloc[-2]) > 1:
        pp = 'percentage points'
    else:
        pp = 'percentage point'
    k_pr = f"{abs(df['W271RC'].iloc[-2]):.2f} {pp}"
t_lt = cont_subt(df['indirect'].iloc[-1])
t_pr = cont_subt(df['indirect'].iloc[-2]).replace(' from', '').replace(' to', '')
    
text = (f'{text1}In the latest quarter, labor income {l_lt} overall growth, '+
        f'following a {l_pr} in {prdt}. Profit income {k_lt} in '+
        f'{ltdt} and {k_pr} in {prdt}. Changes in indirect tax revenue '+
        f'and surpluses {t_lt} aggregate income growth in the latest quarter and '+
        f'{t_pr} in {prdt}. ')
print(text) 

write_txt(text_dir / 'gdi_growth_comp.txt', text)

In the third quarter of 2020, gross domestic income increased at an annual rate of 24.1 percent, following a decrease of 32.6 percent in 2020 Q2 and a decrease of 2.5 percent in 2020 Q1. In the latest quarter, labor income contributed 9.22 percentage points to overall growth, following a subtraction of 10.73 percentage points in 2020 Q2. Profit income contributed 15.53 percentage points in the third quarter of 2020 and subtracted 4.23 percentage points in 2020 Q2. Changes in indirect tax revenue and surpluses subtracted 0.80 percentage point from aggregate income growth in the latest quarter and subtracted 18.33 percentage points in 2020 Q2. 


### Consumer Spending Overview (Levels)

In [21]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

n = {'DPCERC': 'Total',
     'DGDSRC': 'Goods',
     'DSERRC': 'Services',
     'DHUTRC': 'Housing',
     'A011RC': 'ResInv',
     'DMOTRC': 'MotorVeh',
     'DFDHRC': 'Furn',
     'DREQRC': 'RecDG',
     'DFXARC': 'Groc',
     'DCLORC': 'Cloth',
     'DHLCRC': 'Health',
     'DTRSRC': 'Transp',
     'DRCARC': 'RecSer',
     'DFSARC': 'FoodAcc',
     'DIFSRC': 'FinIns'}
n2 = {k[:-2] + 'RA': v for k, v in n.items()}
s = list(n.keys())
s2 = list(n2.keys())

othserv = lambda x: x['Services'] - x['Housing']
shelter = lambda x: x['Housing'] + x['ResInv']

df = (nipa_df(retrieve_table('T10505')['Data'], s)
      .rename(n, axis=1))
df2 = (nipa_df(retrieve_table('T10503')['Data'], s2)
       .rename(n2, axis=1))
real = ((df2 / df2.iloc[-1]) * df.iloc[-1]).assign(OTHSERV = othserv, SHELTER = shelter)
pp = real.divide(pop, axis=0)

keep_col = ['Goods', 'Services', 'Housing', 'ResInv', 'OTHSERV', 'SHELTER']
pp.loc['1989':, keep_col].to_csv(data_dir / 'pce_levels.csv', index_label='date')

lttot = real.Total.iloc[-1] / 1_000_000
prtot = real.Total.iloc[-2] / 1_000_000
pctot = real.Total.loc['2019-10-01'] / 1_000_000
ltdate = dtxt(real.index[-1])['qtr1']
prdate = dtxt(real.index[-2])['qtr1']
totpp = pp.Total.iloc[-1] * 1_000
goodpp = pp.Goods.iloc[-1] * 1_000
servpp = pp.Services.iloc[-1] * 1_000
goodpppc = pp.Goods.loc['2019-10-01'] * 1_000
servpppc = pp.Services.loc['2019-10-01'] * 1_000


text = ('Total consumer spending is '+
        f'\${lttot:.1f} trillion in {ltdate}, compared to a price-adjusted '+
        f'\${prtot:.1f} trillion in {prdate} and \${pctot:.1f} trillion in 2019 Q4. '+
        'On a per person basis, consumer spending is '+
        f'\${totpp:,.0f} in {ltdate}, of which \${goodpp:,.0f} are spent on goods '+
        '(see {\color{red}\\textbf{---}}) and '+
        f'\${servpp:,.0f} on services '+
        '(see {\color{orange}\\textbf{---}}). In the fourth quarter of 2019, '+
        f'before the pandemic, consumer spending on goods was \${goodpppc:,.0f} '+
        f'per person, and spending on services was \${servpppc:,.0f} per person, '+
        'after adjusting for inflation. ')
write_txt(text_dir / 'pce_levels.txt', text)
print(text)

hult = pp['Housing'].iloc[-1] * 1_000
hupc = pp['Housing'].loc['2019-10-01'] * 1_000
rfilt = pp['ResInv'].iloc[-1] * 1_000
rfipc = pp['ResInv'].loc['2019-10-01'] * 1_000

text = ('Within consumer spending on services, housing and utilities spending '+
        f'totals \${hult:,.0f} on an annualized and per person basis in {ltdate} '+
        '(see {\color{green!60!blue}\\textbf{---}}) '+
        f'and \${hupc:,.0f} in 2019 Q4. Construction or improvement '+
        'of housing is considered residential fixed investment, not '+
        'consumer spending, but can be combined with spending to analyze '+
        'patterns in shelter costs. In '+
        f'{ltdate}, residential investment totals \${rfilt:,.0f} per person '+
        '(see {\color{blue!80!black}\\textbf{---}}), '+
        f'compared to \${rfipc:,.0f} in the pre-COVID data covering 2019 Q4. ')
write_txt(text_dir / 'pce2_levels.txt', text)
print('\n', text)

othlt = pp['OTHSERV'].iloc[-1] * 1_000
othpr = pp['OTHSERV'].iloc[-2] * 1_000
othpc = pp.loc['2019-10-01', 'OTHSERV'] * 1_000
chval = ((othlt / othpc) - 1) * 100
chtxt = value_text(chval)
shellt = pp['SHELTER'].iloc[-1] * 1_000
shelpr = pp['SHELTER'].iloc[-2] * 1_000
shelpc = pp.loc['2019-10-01', 'SHELTER'] * 1_000
shelmax = pp['SHELTER'].max() * 1_000
shelmaxdt = dtxt(pp.SHELTER.idxmax())['qtr2']


text = ('Consumer spending on services other than housing and utilities totals \$'+
        f'{othlt:,.0f} per person, on an annaulized basis, in {ltdate} '+
        '(see {\color{blue!75!white}\\textbf{---}}), '+
        f'compared to \${othpr:,.0f} in {prdate}, and \${othpc:,.0f} in '+
        f'2019 Q4. Spending on non-housing services has {chtxt} since 2019 Q4. '+
        'Shelter costs, which combine housing, utilities, and residential fixed '+
        f'investment, are \${shellt:,.0f} per person in {ltdate} '+
        '(see {\color{green!85!blue}\\textbf{---}})'+
        f', \${shelpr:,.0f} '+
        f'in {prdate}, and \${shelpc:,.0f} in 2019 Q4. Shelter spending peaked at '+
        f'\${shelmax:,.0f} per person in {shelmaxdt}, during the housing bubble.')
write_txt(text_dir / 'pce3_levels.txt', text)
print('\n', text)

Total consumer spending is \$14.5 trillion in 2020 Q4, compared to a price-adjusted \$14.5 trillion in 2020 Q3 and \$14.9 trillion in 2019 Q4. On a per person basis, consumer spending is \$43,957 in 2020 Q4, of which \$14,687 are spent on goods (see {\color{red}\textbf{---}}) and \$29,270 on services (see {\color{orange}\textbf{---}}). In the fourth quarter of 2019, before the pandemic, consumer spending on goods was \$13,815 per person, and spending on services was \$31,573 per person, after adjusting for inflation. 

 Within consumer spending on services, housing and utilities spending totals \$8,515 on an annualized and per person basis in 2020 Q4 (see {\color{green!60!blue}\textbf{---}}) and \$8,454 in 2019 Q4. Construction or improvement of housing is considered residential fixed investment, not consumer spending, but can be combined with spending to analyze patterns in shelter costs. In 2020 Q4, residential investment totals \$2,991 per person (see {\color{blue!80!black}\textbf{-

In [22]:
n = {'Total': 'Total',
     'Goods': '\hspace{0.1mm} {\color{red}\\textbf{---}} Goods',
     'MotorVeh': '\hspace{5mm} Motor Vehicles and Parts',
     'Furn': '\hspace{5mm} Furniture and HH Equipment',
     'RecDG': '\hspace{5mm} Recreational Durable Goods',
     'Groc': '\hspace{5mm} Groceries',
     'Cloth': '\hspace{5mm} Clothes and Shoes',
     'OTHSERV': '\hspace{0.1mm} {\color{blue!75!white}\\textbf{---}} Services ex. Shelter',
     'Health': '\hspace{5mm} Health Care Services',
     'Transp': '\hspace{5mm} Transportation',
     'RecSer': '\hspace{5mm} Recreational',
     'FoodAcc': '\hspace{5mm} Food and Accommodations',
     'FinIns': '\hspace{5mm} Financial and Insurance',
     'SHELTER': '\hspace{0.1mm} {\color{green!85!blue}\\textbf{---}} Shelter ',
     'Housing': '\hspace{5mm} Housing Services and Utilities ',
     'ResInv': '\hspace{5mm} Residential Fixed Investment'}

res = pp[n.keys()].rename(n, axis=1) * 1_000
lt = res.iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.loc['2019-10-01']
pr.name = dtxt(pr.name)['qtr1']
p00 = res.loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'pce_levels.tex', sep='&', line_terminator='\\\ ', quotechar=' ')
print(table)

                                                     2020 Q4 2019 Q4 2000 Q1  \
Total                                               \$43,957  45,378  33,883   
\hspace{0.1mm} {\color{red}\textbf{---}} Goods        14,687  13,815   9,148   
\hspace{5mm} Motor Vehicles and Parts                  1,776   1,678   1,437   
\hspace{5mm} Furniture and HH Equipment                1,222   1,126     546   
\hspace{5mm} Recreational Durable Goods                1,583   1,305     264   
\hspace{5mm} Groceries                                 3,445   3,257   2,781   
\hspace{5mm} Clothes and Shoes                         1,151   1,165     850   
\hspace{0.1mm} {\color{blue!75!white}\textbf{--...    20,755  23,119  17,728   
\hspace{5mm} Health Care Services                      7,398   7,792   5,144   
\hspace{5mm} Transportation                            1,103   1,470   1,322   
\hspace{5mm} Recreational                              1,225   1,837   1,450   
\hspace{5mm} Food and Accommodations    

### Consumer Spending and Residential Fixed Investment

In [23]:
s = ['DNPIRY', 'DSERRY', 'DPCERY', 'DGDSRY', 'DMOTRY',
     'DFDHRY', 'DREQRY', 'DFXARY', 'DCLORY', 'DHLCRY',
     'DTRSRY', 'DRCARY', 'DFSARY', 'DIFSRY', 'DHUTRY',
     'A011RY']

n = {'TOTAL': '& Total',
     'DGDSRY': '\cbox{red} & Goods',
     'DMOTRY': '& \hspace{1mm} Motor Vehicles and Parts',
     'DFDHRY': '& \hspace{1mm} Furniture and HH Equipment',
     'DREQRY': '& \hspace{1mm} Recreational Durable Goods',
     'DFXARY': '& \hspace{1mm} Groceries',
     'DCLORY': '& \hspace{1mm} Clothes and Shoes',
     'OTHSERV': '\cbox{blue!75!white} & Services (ex. Shelter)',
     'DHLCRY': '& \hspace{1mm} Health Care Services',
     'DTRSRY': '& \hspace{1mm} Transportation',
     'DRCARY': '& \hspace{1mm} Recreational',
     'DFSARY': '& \hspace{1mm} Food and Accommodations',
     'DIFSRY': '& \hspace{1mm} Financial and Insurance',
     'SHELTER': '\cbox{green!85!blue} & Shelter ',
     'DHUTRY': '& \hspace{1mm} Housing Services and Utilities ',
     'A011RY': '& \hspace{1mm} Residential Fixed Investment'}

total = lambda x: x['DPCERY']
othserv = lambda x: x['DSERRY'] - x['DHUTRY']
shelter = lambda x: x['DHUTRY'] + x['A011RY']

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      .assign(TOTAL = total, OTHSERV = othserv, SHELTER = shelter)
      [list(n.keys())])

# Save timseries to csv
(df.loc['1989':, ['DGDSRY', 'OTHSERV', 'SHELTER']]
   .to_csv(data_dir / 'pce.csv', index_label='date', float_format='%g'))

# Save table
data = df.iloc[-5:].iloc[::-1].T

cols = [f'& {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = df.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = df.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = df.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
(data.round(2).applymap('{:,.2f}'.format)
 .to_csv(data_dir / 'pce.tex', sep='&', line_terminator='\\\ ', quotechar=' '))

ld = dtxt(df.index[-1])['qtr1']
prd = dtxt(df.index[-2])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']

totlt = df['TOTAL'].iloc[-1]
totltt = value_text(totlt, style='contribution_to', ptype='pp', digits=1)
totpr = df['TOTAL'].iloc[-2]
totprt = value_text(totpr, style='contribution', ptype='pp', digits=1)
totpc = df.loc['2019-10-01', 'TOTAL']
totpct = value_text(totpc, style='contribution_of', ptype='pp', digits=1, casual=True)

txt1 = (f'These categories {totltt} GDP growth in {ld} and {totprt} in {prd}, compared to '+
        f'{totpct} in 2019 Q4, before the pandemic.')
write_txt(text_dir / 'pce1.txt', txt1)
print(txt1)

gdslt = df['DGDSRY'].iloc[-1]
gdsltt = value_text(gdslt, style='contribution_to', ptype='pp', digits=1)
serlt = df['OTHSERV'].iloc[-1]
serltt = value_text(serlt, style='contribution', ptype='pp', digits=1, casual=True)
shelt = df['SHELTER'].iloc[-1]
sheltt = value_text(shelt, style='contribution', ptype='pp', digits=1, casual=True)

txt2 = (f'In the {ld2}, household spending on goods {gdsltt} GDP growth, '+
         'household spending on services other than housing and utilities '+
        f'{serltt}, and shelter spending and investment {sheltt}.')
write_txt(text_dir / 'pce2.txt', txt2)
print('\n', txt2)

These categories contributed 1.6 percentage points to GDP growth in 2020 Q4 and contributed 25.4 percentage points in 2020 Q3, compared to an addition of 1.1 percentage points in 2019 Q4, before the pandemic.

 In the the fourth quarter of 2020, household spending on goods subtracted 0.2 percentage point from GDP growth, household spending on services other than housing and utilities added 1.8 percentage points, and shelter spending and investment added 1.4 percentage points.


### Trade Contribution to GDP

In [24]:
s = ['A019RY', 'A253RY', 'A646RY', 'A255RY', 'A656RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'nx.csv', index_label='date')

sl = [('A253RY', 'main'), ('A646RY', 'end'), 
      ('A255RY', 'main'), ('A656RY', 'end')]

d = {}
for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)
    
ldate = dtxt(df.index[-1])['qtr2']

text = (f"Goods exports {d['A253RY']} GDP growth in {ldate} while "+
        f"services exports {d['A646RY']}. Good imports {d['A255RY']} "+
        f"GDP growth and services imports {d['A656RY']}.")

write_txt(text_dir / 'trade.txt', text)

print(text)

Goods exports contributed 1.86 percentage points to GDP growth in the fourth quarter of 2020 while services exports contributed 0.13 percentage point. Good imports subtracted 3.10 percentage points from GDP growth and services imports subtracted 0.45 percentage point.


### Business investment

In [25]:
s = ['Y001RY', 'A009RY', 'Y033RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'businv.csv', index_label='date')

In [26]:
s = ['Y001RC', 'B009RC', 'Y033RC', 'A008RC']
data = nipa_df(retrieve_table('T10105')['Data'], s)
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
df1 = data.divide(gdp, axis=0) * 100
(df1.loc['1989':, ['Y001RC', 'B009RC', 'Y033RC']]
    .to_csv(data_dir / 'businvsh.csv', index_label='date'))

ltdate = dtxt(df1.index[-1])['qtr1']

levels = {}
shgdp = {}
for series in ['Y001RC', 'B009RC', 'Y033RC']:
    level_val = data[series].iloc[-1] / 1_000
    txt = f"\${level_val:,.0f} billion"
    levels[series] = txt
    shgdp[series] = f"{df1[series].iloc[-1]:.1f} percent of GDP"

text = ('Business investments in fixed assets are grouped into three '+
        'categories: structures, equipment, and intellectual property '+
        '(for example software and R\&D). Investment in structures was '+
        f'{levels["B009RC"]} in {ltdate}, equivalent to {shgdp["B009RC"]} '+
        '(see {\color{yellow!50!orange}\\textbf{---}}). '+
        f'Equipment investment was {levels["Y033RC"]} or {shgdp["Y033RC"]} '+
        '(see {\color{cyan!60!white}\\textbf{---}}), '+
        'and intellectual property investment '+
        f'was {levels["Y001RC"]} or {shgdp["Y001RC"]} '+
        '(see {\color{violet}\\textbf{---}}). ')
write_txt(text_dir / 'businv_sh.txt', text)
print(text)

Business investments in fixed assets are grouped into three categories: structures, equipment, and intellectual property (for example software and R\&D). Investment in structures was \$559 billion in 2020 Q4, equivalent to 2.6 percent of GDP (see {\color{yellow!50!orange}\textbf{---}}). Equipment investment was \$1,266 billion or 5.9 percent of GDP (see {\color{cyan!60!white}\textbf{---}}), and intellectual property investment was \$1,056 billion or 4.9 percent of GDP (see {\color{violet}\textbf{---}}). 


In [27]:
s = ['W790RC', 'W276RC', 'W987RC']
df = nipa_df(retrieve_table('T50100')['Data'], s).join(data['A008RC'])
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = (df.div(gdp, axis=0) * 100).dropna()
res.loc['1989':].to_csv(data_dir / 'businv_main.csv', index_label='date')

ltdate1 = dtxt(df.index[-1])['qtr1']
ltdate2 = dtxt(df.index[-1])['qtr2']

levels = {}
shgdp = {}
for series in ['W790RC', 'W276RC', 'W987RC', 'A008RC']:
    level_val = df[series].iloc[-1] / 1_000
    n = ''
    if level_val < 0:
        n = 'negative '
        level_val = abs(level_val)
    txt = f"{n}\${level_val:,.0f} billion"
    levels[series] = txt
    shgdp[series] = f"{res[series].iloc[-1]:.1f} percent of GDP"
    
diff = ((df['W987RC'].iloc[-1] / df.loc['2019-10-01', 'W987RC']) - 1) * 100
gdt = value_text(diff)
diff2 = ((df['W790RC'].iloc[-1] / df.loc['2019-10-01', 'W790RC']) - 1) * 100
if diff2 < -95:
    ndt = 'but collapsed completely'
else:
    ndt = 'and ' + value_text(diff2)
gpcl = f"{n}\${df.loc['2019-10-01', 'W987RC'] / 1_000:,.0f} billion"
npcl = f"{n}\${df.loc['2019-10-01', 'W790RC'] / 1_000:,.0f} billion"
if diff < 0:
    txt = 'as gross investment fell while deprecation was relatively constant'
else:
    txt = 'as gross investment recovered from its pandemic lows'
    
text = (f'In the {ltdate2}, gross private business investment totals '+
        f'{levels["W987RC"]} on a seasonally-adjusted annualized basis, '+
        f'equivalent to {shgdp["W987RC"]} '+
        '(see {\color{blue!60!violet}\\textbf{---}}). Private business investment '+
        f'in fixed assets totals {levels["A008RC"]}, or {shgdp["A008RC"]} '+
        '(see {\color{cyan!80!white}\\textbf{---}}). Private business depreciation '+
        f'totals {levels["W276RC"]} in the quarter, or {shgdp["W276RC"]} '+
        '(see {\color{magenta}\\textbf{---}}). '+
        f'As a result, net investment is {levels["W790RC"]}, or {shgdp["W790RC"]} '+
        '(see {\color{green!80!blue}\\textbf{---}}). \n\n '+
        'In 2019 Q4, prior to the COVID-19 pandemic, private business gross '+
        f'investment was {gpcl}. Since 2019 Q4, annualized gross investment {gdt}. '+
        f'Net investment was {npcl} in 2019 Q4, {ndt} from 2019 Q4 to {ltdate1}, '+
        f'{txt}. ')
write_txt(text_dir / 'businv_levels.txt', text)
print(text)  

In the the fourth quarter of 2020, gross private business investment totals \$2,864 billion on a seasonally-adjusted annualized basis, equivalent to 13.3 percent of GDP (see {\color{blue!60!violet}\textbf{---}}). Private business investment in fixed assets totals \$2,881 billion, or 13.4 percent of GDP (see {\color{cyan!80!white}\textbf{---}}). Private business depreciation totals \$2,355 billion in the quarter, or 11.0 percent of GDP (see {\color{magenta}\textbf{---}}). As a result, net investment is \$509 billion, or 2.4 percent of GDP (see {\color{green!80!blue}\textbf{---}}). 

 In 2019 Q4, prior to the COVID-19 pandemic, private business gross investment was \$2,839 billion. Since 2019 Q4, annualized gross investment increased 0.9 percent. Net investment was \$569 billion in 2019 Q4, and decreased 10.4 percent from 2019 Q4 to 2020 Q4, as gross investment recovered from its pandemic lows. 


### Durable goods new orders

In [30]:
# New orders for capital goods excluding defense or aircraft
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/advm3'
param = 'cell_value,time_slot_id'
t = '&time=from+1992'
cat = '&category_code=NXA'
dtc = '&data_type_code=NO'
oth = '&for=us&seasonally_adj=yes'
url = f'{base}?get={param}&key={key}{cat}{dtc}{t}{oth}'

r = requests.get(url).json()

df = pd.DataFrame({'date': [pd.to_datetime(i[4]) for i in r[1:]], 
                   'value': [float(i[0]) for i in r[1:]]}).sort_values('date')

df = df.set_index('date')

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])

result = ((df['value'].resample('QS').sum() * 4  / gdp['A191RC']).dropna() * 100).iloc[1:]
(result.rename('value').loc['1989':].to_csv(data_dir / 'dgno.csv', index_label='date', header=True))

node = end_node(result, 'purple!50!violet')
write_txt(text_dir / 'dgno_node.txt', node)

ldate = dtxt(df.index[-1])['mon1']
date_latest = dtxt(df.index[-1])['datetime']
month_short = df.index[-1].strftime('%b')

In [31]:
one_yr_ch = (df.pct_change(12) * 100).dropna()['value'].iloc[-1]
val = inc_dec_percent(one_yr_ch)

text = ('New orders for manufactured core capital goods excluding aircraft '+
        f'totalled \${df.iloc[-1][0] / 1000:,.0f} billion in {ldate}, '+
        f'equivalent to {result.iloc[-1]:.1f} percent of GDP '+
        '(see {\color{purple!50!violet}\\textbf{---}}). New orders '+
        f'{val} over the past year.')

write_txt(text_dir / 'dgno.txt', text)
print(text)

text = ('xtick={{1992-01-01}, {1995-01-01}, {2000-01-01}, {2005-01-01}, '+
f'{{2010-01-01}}, {{2015-01-01}}, {{{date_latest}}}}}, '+
f'xticklabels={{`92, `95, `00, `05, `10, `15, {month_short}}}, ')

text_full = ('\\begin{tikzpicture}'+
'\\begin{axis}[\\bbar{y}{0}, \dateaxisticks ytick={4, 6, 8}, width=6.4cm, height=5.4cm,'+
'ymin=3.0, clip=false,'+
text + 
'minor xtick={}]'+
'\\rebars'+
'\\thickline{purple!50!violet}{date}{value}{data/dgno.csv}'+
'\input{text/dgno_node.txt}'+
'\end{axis}'+
'\end{tikzpicture}')

write_txt(text_dir / 'dgno.tex', text_full)

New orders for manufactured core capital goods excluding aircraft totalled \$73 billion in January 2021, equivalent to 4.0 percent of GDP (see {\color{purple!50!violet}\textbf{---}}). New orders increased by 9.1 percent over the past year.


### Government spending and investment

In [32]:
n = {'A822RY': 'Total',
     'A823RY': '\hspace{1mm}Federal total',
     'A824RY': '\hspace{1mm}\cbox{blue!60!black}National defense',
     'A997RY': '\hspace{7mm}Consumption expenditures',
     'A788RY': '\hspace{7mm}Gross investment',
     'A825RY': '\hspace{1mm}\cbox{green!85!black}Nondefense',
     'A542RY': '\hspace{7mm}Consumption expenditures',
     'A798RY': '\hspace{7mm}Gross investment',
     'A829RY': '\hspace{-2mm}\cbox{purple!70!magenta}State \& local total',
     'A991RY': '\hspace{5mm}Consumption expenditures',
     'A799RY': '\hspace{5mm}Gross investment'}

s = n.keys()

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'gov.csv', index_label='date')

d = series_info(df['A822RY'])    
d2 = {}
for i in ['A824RY', 'A825RY', 'A829RY']:
    d2[i] = cont_subt(df[i].iloc[-1], 'end')
    
ltmain = cont_subt(d['val_latest'], 'main')
compare = compare_text(d['val_latest'], d['one_year_mean'], [0.05, 0.25, 0.5])
compare2 = compare_text(d['val_latest'], d['mean'], [0.05, 0.25, 0.5])

ltoym = f'{d["one_year_mean"]:.2f} ' + ['percentage point' if abs(d["one_year_mean"]) < 1.05 else 'percentage points'][0]
ltmean = f'{d["mean"]:.2f} ' + ['percentage point' if abs(d["mean"]) < 1.05 else 'percentage points'][0]
    
txt = ('Government consumption expeditures and gross investment, which provide '+
       f'services and infrastructure, {ltmain} real GDP growth in {d["date_latest_ft"]}, '+
       f'{compare} the average contribution of {ltoym} '+
       f'over the past year, and {compare2} the average of {ltmean} since 1989. '+
       f'In {d["date_latest_ft"]}, federal defense spending and investment (see\cbox{{blue!60!black}}) '+
       f'{d2["A824RY"]}, nondefense federal government spending and investment '+
       f'(see\cbox{{green!85!black}}) {d2["A825RY"]}, and state and '+
       f'local government (see\cbox{{purple!70!magenta}}) {d2["A829RY"]}.')
write_txt(text_dir / 'gov.txt', txt)
print(txt)

Government consumption expeditures and gross investment, which provide services and infrastructure, subtracted 0.19 percentage point from real GDP growth in 2020 Q4, slightly below the average contribution of 0.01 percentage point over the past year, and substantially below the average of 0.23 percentage point since 1989. In 2020 Q4, federal defense spending and investment (see\cbox{blue!60!black}) contributed 0.20 percentage point, nondefense federal government spending and investment (see\cbox{green!85!black}) subtracted 0.26 percentage point, and state and local government (see\cbox{purple!70!magenta}) subtracted 0.13 percentage point.


In [33]:
result = df[n.keys()]

data = result.iloc[-5:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = result.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
data = data.applymap('{:.2f}'.format)
data.to_csv(data_dir / 'gov.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

### Government receipts and expenditures

In [34]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

d = {'slggdp': {'name': 'combined state and local government', 
                'table': 'T30300', 'series': 
                {'W024RC': 'SLG_EXP', 'W023RC': 'SLG_REC'}},
     'fedgdp': {'name': 'federal government', 
                'table': 'T30200', 'series':
                {'W005RC': 'FED_REC', 'W013RC': 'FED_EXP'}}}

data = pd.DataFrame()

ltdate = dtxt(gdp.index[-1])['qtr1']

for group, details in d.items():
    df = nipa_df(retrieve_table(details['table'])['Data'], details['series'].keys())
    (df.div(gdp, axis=0) * 100).loc['1989':].to_csv(data_dir / f'{group}.csv', 
                                                    index_label='date', float_format='%g')
    
    grp = group[:3].upper()
    df.columns = [details['series'][i] for i in df.columns]
    bal = df[f'{grp}_REC'] - df[f'{grp}_EXP']
    bal_lt = bal.dropna().iloc[-1]
    bal_lt_dt = dtxt(bal.dropna().index[-1])['qtr1']
    def_sur = 'deficit' if bal_lt < 0 else 'surplus'
    df[f'{grp}_BAL'] = bal
    
    for col in df.columns:
        data[col] = df[col]
        data[col+'_GDP'] = df[col].div(gdp, axis=0) * 100
        
    bal_gdp = data[f'{grp}_BAL_GDP'].dropna().iloc[-1]
    bal_txt = (f'In {bal_lt_dt}, the {d[group]["name"]} {def_sur} was '+
               f'\${abs(bal_lt) / 1000:,.0f} billion or {abs(bal_gdp):.1f} percent of GDP. ')
    
    exp_txt = (f'{d[group]["name"].capitalize()} expenditures total '+
               f'\${data[grp+"_EXP"].iloc[-1] / 1000000:.1f} trillion, '+
               f'or {data[grp+"_EXP_GDP"].iloc[-1]:.1f} percent of GDP, in {ltdate}. ')
    
        
    if pd.isna(df[[i for i in df.columns if i[4:] == 'REC'][0]].iloc[-1]) == True:
        rec_txt = (f'BEA has not yet released receipts data for {ltdate}, however, '+
                   f'in {bal_lt_dt}, {d[group]["name"]} receipts total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion, '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    else:
        rec_txt = ('Receipts for the same period total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    
    text = exp_txt + rec_txt + bal_txt
    write_txt(text_dir / f'{group}.txt', text)
    print(text)

Combined state and local government expenditures total \$3.0 trillion, or 13.9 percent of GDP, in 2020 Q4. BEA has not yet released receipts data for 2020 Q4, however, in 2020 Q3, combined state and local government receipts total \$2.9 trillion, or 13.8 percent of GDP. In 2020 Q3, the combined state and local government deficit was \$77 billion or 0.4 percent of GDP. 
Federal government expenditures total \$6.0 trillion, or 28.0 percent of GDP, in 2020 Q4. BEA has not yet released receipts data for 2020 Q4, however, in 2020 Q3, federal government receipts total \$3.7 trillion, or 17.4 percent of GDP. In 2020 Q3, the federal government deficit was \$3,529 billion or 16.7 percent of GDP. 


### Public Debt by Holder

In [10]:
series = ['FDHBATN', 'GFDEBTN', 'FDHBFRBN', 'FDHBPIN', 'FDHBFIN']
start = '1988-01-01'
ftype = '&file_type=json'
base = 'https://api.stlouisfed.org/fred/series/observations?'

df = pd.DataFrame()

for srs in series:
    param = f'series_id={srs}&observation_start={start}&api_key={fred_key}'
   

    url = f'{base}{param}{ftype}'
    r = requests.get(url).json()['observations']
    data = pd.Series({pd.to_datetime(i['date']): (float(i['value']) / 1000.0) 
                      if srs in series[:2] 
                      else float(i['value']) for i in r})
    
    df[srs] = data

In [11]:
df = df.dropna()
df['PD'] = df['FDHBPIN'] - df['FDHBFIN']
df['IG'] = df['GFDEBTN'] - (df['FDHBFRBN'] + df['FDHBPIN'])

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = df.div(gdp / 1000.0, axis=0).dropna()

(data[['PD', 'FDHBFIN', 'FDHBFRBN', 'IG']] * 100).to_csv(data_dir / 'pubdebt.csv', index_label='date')

ld = pd.to_datetime(df.index[-1])
ldate = dtxt(ld)['qtr2']

sh = df.div(df['GFDEBTN'], axis=0).iloc[-1] * 100
lv = df.iloc[-1] / 1000
dl = data.iloc[-1] * 100

text = (f'In {ldate}, total public debt was \${lv.GFDEBTN:.1f} trllion, '+
        f'equivalent to {dl.GFDEBTN:.1f} percent of GDP. Of this, \${lv.PD:.1f} '+
        f'trillion, or {sh.PD:.1f} percent of the total, is held by '+
        'private domestic investors (see\cbox{green!60!black}). An additional '+
        f'\${lv.FDHBFIN:.1f} trillion, or {sh.FDHBFIN:.1f} percent '+
        'of the total, is held by foreign investors (see\cbox{orange!70!white}). '+
        'The remainder is held by the Federal Reserve (see\cbox{blue}) '+
        'and various government agencies and trusts (see\cbox{cyan!50!white}), '+
        'such as the Social Security Trust Fund. ')
write_txt(text_dir / 'pubdebt.txt', text)
print(text)

In the third quarter of 2020, total public debt was \$26.9 trllion, equivalent to 127.3 percent of GDP. Of this, \$9.5 trillion, or 35.3 percent of the total, is held by private domestic investors (see\cbox{green!60!black}). An additional \$7.1 trillion, or 26.2 percent of the total, is held by foreign investors (see\cbox{orange!70!white}). The remainder is held by the Federal Reserve (see\cbox{blue}) and various government agencies and trusts (see\cbox{cyan!50!white}), such as the Social Security Trust Fund. 


### Personal Income

In [37]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A065RC', 'A033RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC', 'A061RC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'])
      .drop(['A061RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
growth_contrib(df, 'A065RC').loc['1989':].to_csv(data_dir / 'pi.csv', index_label='date')


data = growth_contrib(df, 'A065RC').rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1)

val3y = data.rolling(12).mean().iloc[-1]

ltdate = dtxt(data.index[-1])['qtr1']

d = {}
for i in data.keys():
    val = data[i].iloc[-1]
    pp = 'percentage points' if abs(val) > 1.05 else 'percentage point'
    if val >= 0.1:
        d[i] = f'contributed {val:.2f} {pp} to'
    elif val <= 0.1:
        d[i] = f'subtracted {abs(val):.2f} {pp} from'
    else:
        d[i] = 'did not contribute significantly to'
        
d['TOTAL'] = (d['TOTAL']
              .replace('contributed', 'increased at an annualized rate of')
              .replace('subtracted', 'decreased at an annualized rate of')
              .replace('percentage points', 'percent')
              .replace(' to', '').replace(' from', ''))

for i in ['TRANSFER', 'CAPITAL']:
    d[i] = d[i].replace(' to', '').replace(' from', '')
    
d2 = {}
for i in data.keys():
    val = val3y[i]
    pp = 'percentage points' if abs(val) > 1.05 else 'percentage point'
    if val >= 0.1:
        d2[i] = f'contributed an average of {val:.2f} {pp}'
    elif val <= 0.1:
        d2[i] = f'subtracted an average of {abs(val):.2f} {pp}'
    else:
        d2[i] = 'did not contribute significantly, on average'
        
text = (f'Aggregate real personal income {d["TOTAL"]} in {ltdate}. '+
        f'Labor income {d["LABOR"]} overall growth, '+
        f'capital income {d["CAPITAL"]}, and welfare income {d["TRANSFER"]}. ')
write_txt(text_dir / 'pi.txt', text)
print(text)

Aggregate real personal income decreased at an annualized rate of 8.29 percent in 2020 Q4. Labor income contributed 3.82 percentage points to overall growth, capital income subtracted 0.32 percentage point, and welfare income subtracted 11.80 percentage points. 


### Government Personal Income

In [38]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['A063RC', 'B202RC', 'A061RC', 'W055RC']
df = nipa_df(retrieve_table('T20100')['Data'], s)

n = {'A063RC': 'Welfare',
     'B202RC': 'WandS'}
result = df.multiply(deflator, axis=0).divide(population, axis=0)
result['TaxSI'] = result['A061RC'] + result['W055RC']
result = result.drop(['A061RC', 'W055RC'], axis=1).rename(n, axis=1)
result.loc['1989':].to_csv(data_dir / 'govpi.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = '2019-10-01'
prdate = dtxt(pd.to_datetime(prdt))['qtr1']
wsltval = result['WandS'].iloc[-1] * 1_000
wsprval = result.loc[prdt, 'WandS'] * 1_000
wltval = result['Welfare'].iloc[-1] * 1_000
wprval = result.loc[prdt, 'Welfare'] * 1_000
w89val = result.loc['1989-01-01', 'Welfare'] * 1_000
tltval = result['TaxSI'].iloc[-1] * 1_000
tprval = result.loc[prdt, 'TaxSI'] * 1_000
t89val = result.loc['1989-01-01', 'TaxSI'] * 1_000

text = (f'In {ltdate}, government worker wages and salaries, not including '+
        f'benefits, were equivalent to \${wsltval:,.0f} per capita, following a '+
        f'price-adjusted \${wsprval:,.0f} in {prdate} '+
        '(see {\color{orange}\\textbf{---}}). Welfare payments were equivalent '+
        f'to \${wltval:,.0f} per capita in {ltdate}, compared to \${wprval:,.0f} '+
        f'per capita in {prdate} '+
        '(see {\color{violet}\\textbf{---}}). In 1989 Q1, welfare payments were '+
        f'equivalent to \${w89val:,.0f} per person.  \n\n Personal current taxes '+
        f'and social insurance contributions total \${tltval:,.0f} per capita '+
        f'in {ltdate}, \${tprval:,.0f} in {prdate}, and \${t89val:,.0f} in 1989 '+
        '(see {\color{cyan!80!blue}\\textbf{---}}).')
write_txt(text_dir / 'govpi.txt', text)
print(text)

In 2020 Q4, government worker wages and salaries, not including benefits, were equivalent to \$4,344 per capita, following a price-adjusted \$4,518 in 2019 Q4 (see {\color{orange}\textbf{---}}). Welfare payments were equivalent to \$11,331 per capita in 2020 Q4, compared to \$9,558 per capita in 2019 Q4 (see {\color{violet}\textbf{---}}). In 1989 Q1, welfare payments were equivalent to \$3,858 per person.  

 Personal current taxes and social insurance contributions total \$11,181 per capita in 2020 Q4, \$11,235 in 2019 Q4, and \$7,067 in 1989 (see {\color{cyan!80!blue}\textbf{---}}).


### Government Consumption and Investment

In [39]:
s = ['A823RC', 'A829RC', 'A191RC', 'A824RC', 'A825RC']
df = nipa_df(retrieve_table('T10105')['Data'], s).sort_index()
result = df.drop('A191RC', axis=1).divide(df['A191RC'], axis=0) * 100
result.loc['1989':].to_csv(data_dir / 'govci.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = pd.to_datetime('2019-10-01')
prdate = dtxt(prdt)['qtr1']
fndnom = df['A825RC'].iloc[-1] / 1_000
fndlt = result['A825RC'].iloc[-1]
fndpr = result.loc[prdt, 'A825RC']
dlt = result['A824RC'].iloc[-1]
dpr = result.loc[prdt, 'A824RC']
d89 = result.loc['1989-01-01', 'A824RC']
slt = result['A829RC'].iloc[-1]
spr = result.loc[prdt, 'A829RC']

text = (f'In {ltdate}, federal non-defense spending and investment '+
        f'was \${fndnom:,.1f} billion, equivalent to '+
        f'{fndlt:.1f} percent of GDP, compared to {fndpr:.1f} percent '+
        f'of GDP in {prdate} '+
        '(see {\color{green!85!black}\\textbf{---}}). '
        'Federal spending on national defense was equivalent to '+
        f'{dlt:.1f} percent of GDP in the latest quarter and {dpr:.1f} percent in '+
        f'{prdate} '+
        '(see {\color{blue!60!black}\\textbf{---}}). '
        f'National defense spending was {d89:.1f} percent of GDP in '+
        '1989 Q1. \n\n In 2020 Q2, state and local government spending '+
        f'and investment was equivalent to {slt:.1f} percent of GDP, compared '+
        f'to {spr:.1f} percent in {prdate} '+
        '(see {\color{purple!70!magenta}\\textbf{---}}).')
write_txt(text_dir / 'govci.txt', text)
print(text)

In 2020 Q4, federal non-defense spending and investment was \$589.0 billion, equivalent to 2.7 percent of GDP, compared to 2.6 percent of GDP in 2019 Q4 (see {\color{green!85!black}\textbf{---}}). Federal spending on national defense was equivalent to 4.2 percent of GDP in the latest quarter and 4.0 percent in 2019 Q4 (see {\color{blue!60!black}\textbf{---}}). National defense spending was 6.9 percent of GDP in 1989 Q1. 

 In 2020 Q2, state and local government spending and investment was equivalent to 10.9 percent of GDP, compared to 10.8 percent in 2019 Q4 (see {\color{purple!70!magenta}\textbf{---}}).


### Consumer Spending Growth Contributions

In [40]:
s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A067RC', 'A068RC', 'A071RC', 'DPCERC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(OTHER = lambda x: -(x['A068RC'] - x['DPCERC']),
              SAVING = lambda x: -x['A071RC'])
      .drop(['A068RC'], axis=1)
      .divide(population, axis=0)
      .multiply(deflator, axis=0))

data = growth_contrib(df, 'DPCERC').rolling(4).mean()
data3y = growth_contrib(df, 'DPCERC').rolling(12).mean()
data.loc['1989':].to_csv(data_dir / 'pcedecomp.csv', index_label='date')

In [41]:
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'

pce = round(data['DPCERC'].iloc[-1], 1)
if pce > 0:
    pcedir = 'increased'
elif pce < 0:
    pcedir = 'decreased'
else: 
    pcedir = ''
pcetext = f'{pcedir} at an average rate of {abs(pce):.1f} percent' 
if pce == 0:
    pcetext = 'was unchanged'
    
slist = ['A067RC', 'SAVING', 'OTHER']
d = {}
for i in slist:
    d[i] = round(data[i].iloc[-1], 1)
    tname = f'{i}txt'
    if d[i] > 0:
        tmpdir = 'added'
    elif d[i] < 0:
        tmpdir = 'subtracted'
    else:
        tmpdir = ''
    tmptxt = f'{tmpdir} {abs(d[i]):.1f} percentage points'
    if d[i] == 0:
        tmptxt = "didn't affect the total"
    d[tname] = tmptxt

In [42]:
pcetxt1 = (f'Real per capita consumer spending {pcetext} over the '+
           f'four quarters ending {date}. Changes to disposable income '+
           f'{d["A067RCtxt"]}, changes to saving {d["SAVINGtxt"]}, and '+
           f'changes to other outlays {d["OTHERtxt"]}.')

pcetxt2 = ('Over the past three years, real per capita consumer spending '+
           f'growth has averaged {data3y["DPCERC"].iloc[-1]:.1f} percent, '+
           f'with income growth contribuing an average of {data3y["A067RC"].iloc[-1]:.1f} '+
           'percentage points and saving subtracting an average of '+
           f'{abs(data3y["SAVING"].iloc[-1]):.1f} percentage points.')

pcetxt = f'{pcetxt1} {pcetxt2}'

write_txt(text_dir / 'pcedecomp.txt', pcetxt)

print(pcetxt1)

print(pcetxt2)

Real per capita consumer spending increased at an average rate of 0.3 percent over the four quarters ending 2020 Q4. Changes to disposable income subtracted 0.3 percentage points, changes to saving subtracted 0.1 percentage points, and changes to other outlays added 0.7 percentage points.
Over the past three years, real per capita consumer spending growth has averaged 1.4 percent, with income growth contribuing an average of 1.5 percentage points and saving subtracting an average of 0.2 percentage points.


### Sectoral Accounts

**NOTE:** Need to convert "deficit", "borrower" etc to parameters!

In [43]:
s = ['W162RC', 'W994RC', 'AD01RC', 'W995RC', 'W996RC', 'AD03RC']
df = (nipa_df(retrieve_table('T50100')['Data'], s).div(
      nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'], axis=0)) * 100

df[['W995RC', 'W996RC', 'AD03RC']].dropna().loc['1989':].to_csv(data_dir / 'sectbal2.csv', index_label='date')

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC']
data = data.dropna()
date = dtxt(data.index[-1])['qtr1']
datepr = '2015-01-01'
dateprtxt = dtxt(pd.to_datetime(datepr))['qtr1']
yrpr = dtxt(pd.to_datetime(datepr))['year']

data.dropna().loc['1989':].to_csv(data_dir / 'sectbal.csv', index_label='date')

priv_curr = abs(data['PRIV'].iloc[-1])
priv_prev = abs(data.loc[datepr, 'PRIV'])
gov_curr = abs(data['GOV'].iloc[-1])
gov_prev = abs(data.loc[datepr, 'GOV'])
row_curr = abs(data['ROW'].iloc[-1])
row_prev = abs(data.loc[datepr, 'ROW'])

compare = compare_text(priv_curr, priv_prev, [0.4, 1.0, 3.5])

sectbaltxt = (f"In {date}, the US private sector was a net lender (running a surplus) of "+
              f"the equivalent of {priv_curr:.1f} percent of GDP, {compare} the {priv_prev:.1f} "+
              f"percent surplus in {dateprtxt}. The rest of the world was a net lender to the US to the "+
              f"equivalent of {row_curr:.1f} percent of GDP in {date}, compared to {row_prev:.1f} "+
              f"percent in {dateprtxt}. Balancing these transactions, the government (federal, state, "+
              f"and local combined) was a net borrower (running a deficit) of the equivalent "+
              f"of {gov_curr:.1f} percent of GDP, compared to {gov_prev:.1f} percent in {yrpr}. ")

write_txt(text_dir / 'sectbal.txt', sectbaltxt)
print(sectbaltxt)

dt = df.dropna().index[-1]
ltdate = dtxt(dt)['qtr1']
hhsh = df.loc[dt, 'W996RC']
pbussh = f"{df.loc[dt, 'W995RC']:.1f} percent of GDP"

if (df.loc[dt, 'W995RC'] > -0.1) & (df.loc[dt, 'W995RC'] < 0.1):
    pbussh = 'unchanged'
    
text = (f'Within the private sector, households were net lenders of the equivalent of {hhsh:.1f} '+
        f'percent of GDP in {ltdate} (see\cbox{{orange!90!yellow}}), while the '+
         'net financial balance of private businesses--corporate and noncorporate--was '+
        f'equivalent to {pbussh} (see\cbox{{purple!50!red}}).')  

write_txt(text_dir / 'sectbal2.txt', text)
print('\n', text)

In 2020 Q3, the US private sector was a net lender (running a surplus) of the equivalent of 14.6 percent of GDP, far above the 2.6 percent surplus in 2015 Q1. The rest of the world was a net lender to the US to the equivalent of 3.3 percent of GDP in 2020 Q3, compared to 2.4 percent in 2015 Q1. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of 17.9 percent of GDP, compared to 5.0 percent in 2015. 

 Within the private sector, households were net lenders of the equivalent of 11.5 percent of GDP in 2020 Q3 (see\cbox{orange!90!yellow}), while the net financial balance of private businesses--corporate and noncorporate--was equivalent to 3.1 percent of GDP (see\cbox{purple!50!red}).


### Current Account Balance

In [44]:
s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

n = {'A1073C': 'Current receipts',
     'B020RC': '\hspace{1mm}Exports',
     'A253RC': '\hspace{3mm}Goods',
     'A332RC': '\hspace{5mm}Durable',
     'A339RC': '\hspace{5mm}Nondurable',
     'A646RC': '\hspace{3mm}Services',
     'B645RC': '\hspace{1mm}Income receipts',
     'LA000035': '\hspace{1mm}Transfer receipts',
     'W163RC': 'Current payments',
     'B021RC': '\hspace{1mm}Imports',
     'A255RC': '\hspace{3mm}Goods',
     'A333RC': '\hspace{5mm}Durable',
     'A340RC': '\hspace{5mm}Nondurable',
     'B656RC': '\hspace{3mm}Services',
     'A655RC': '\hspace{1mm}Income payments',
     'A123RC': '\hspace{1mm}Transfer payments',
     'A124RC': 'Current account balance'}

s = ['A124RC', 'GOODS', 'SERVICES', 'INCOME', 'TRANSFERS']

df = (nipa_df(retrieve_table('T40100')['Data'], n.keys())
      .assign(GOODS = lambda x: x['A253RC'] - x['A255RC'],
              SERVICES = lambda x: x['A646RC'] - x['B656RC'],
              INCOME = lambda x: x['B645RC'] - x['A655RC'],
              TRANSFERS = lambda x: - x['A123RC']))

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0).loc['1989':].multiply(100).round(2))

data[s].dropna().to_csv(data_dir / 'cab.csv', index_label='date')

cab = abs(data.dropna()['A124RC'].iloc[-1])
tb = abs(data.dropna()['GOODS'].iloc[-1])
ld = dtxt(data.dropna().index[-1])['qtr1']
ltdate = dtxt(data.index[-1])['qtr1']
prdate = dtxt(data.index[-2])['qtr1']
lttbval = abs(data['GOODS'].iloc[-1])
tbprval = abs(data['GOODS'].iloc[-2])
prval = abs(data['A124RC'].iloc[-2])


if pd.isna(data['A124RC'].iloc[-1]) == True:
    mtxt = (f'The initial GDP report for {ltdate} does not include the '+
            'data needed to calculate the current account balance, however, '+
            f'the goods trade deficit for {ltdate} is equivalent to '+
            f'{lttbval:.1f} percent of GDP. ')
else:
    mtxt = (f'In {prdate}, the current account deficit was equivalent to '
            f'{prval:.1f}, and the trade deficit was equivalent to '+
            f'{tbprval:.1f} percent of GDP.')

text = (f'As of {ld}, the US runs a current account deficit of {cab:.1f} '+
        'percent of GDP, primarily as the result of a trade deficit on '+
        f'goods of {tb:.1f} percent of GDP. {mtxt}')
write_txt(text_dir / 'cab.txt', text)
print(text)

As of 2020 Q3, the US runs a current account deficit of 3.3 percent of GDP, primarily as the result of a trade deficit on goods of 4.6 percent of GDP. The initial GDP report for 2020 Q4 does not include the data needed to calculate the current account balance, however, the goods trade deficit for 2020 Q4 is equivalent to 4.8 percent of GDP. 


In [45]:
result = data[n.keys()]
data2 = result.iloc[-6:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data2.columns)]

data2.columns = cols
data2['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data2['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data2.index = data2.index.map(n)
data2 = data2.applymap('{:.2f}'.format).replace('nan', '--')
data2.to_csv(data_dir / 'cab.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

### Federal Interest Outlays share of GDP

In [9]:
tot = fred_df('FYOINT')['VALUE']
fy = tot.index[-1].year
pfy = tot.index[-2].year
lttot = tot.iloc[-1] / 1000
prtot = tot.iloc[-2] / 1000

data = fred_df('FYOIGDA188S')['VALUE']
data.index = data.index + pd.DateOffset(months=6)
data.to_csv(data_dir / 'fedintexp.csv', index_label='date', header=True)

fysh = data.index[-1].year
pfysh = data.index[-2].year
latest = data.iloc[-1]
pval = data.iloc[-2]
val90s = data.loc['1990':'1999'].mean()

text = (f'The Office of Management and Budget '+
        '\href{https://www.whitehouse.gov/omb/historical-tables/}{reports} '+
        f'federal interest outlays of \${lttot:.1f} billion in fiscal year {fy}, '+
        f'compared to \${prtot:.1f} billion in fiscal year {pfy}. '+
        f'Put into the context of the size of the economy, federal interest '+
        f'outlays in {fysh} were equivalent to {latest:.1f} percent of '+
        f'GDP, {pval:.1f} percent of GDP in {pfysh}, and an average of {val90s:.1f} '+
        'percent in the 1990s, when interest rates were substantially higher. ')
write_txt(text_dir / 'fedintexp.txt', text)
print(text)

node = end_node3(data, 'magenta')
write_txt(text_dir / 'fedintexp_node.txt', node)

The Office of Management and Budget \href{https://www.whitehouse.gov/omb/historical-tables/}{reports} federal interest outlays of \$344.7 billion in fiscal year 2020, compared to \$375.6 billion in fiscal year 2019. Put into the context of the size of the economy, federal interest outlays in 2020 were equivalent to 1.6 percent of GDP, 1.8 percent of GDP in 2019, and an average of 2.9 percent in the 1990s, when interest rates were substantially higher. 


In [45]:
#nipa_series_codes(retrieve_table('T20100'))

### Corporate Profits Destination

In [47]:
s = ['A032RC', 'A438RC', 'A054RC', 'B056RC', 'A127RC']
cprof = (nipa_df(retrieve_table('T11200')['Data'], s) / 1000000).dropna()

cprof['NNI'] = cprof['A032RC'] - cprof['A438RC']
cprof['TAX'] = cprof['A054RC'] / cprof['NNI'] * 100 
cprof['DIV'] = cprof['B056RC'] / cprof['NNI'] * 100 
cprof['RE'] = cprof['A127RC'] / cprof['NNI'] * 100 

cprof[['TAX', 'DIV', 'RE']].loc['1989':].to_csv(data_dir / 'cprof.csv', index_label='date')

dt = cprof.index[-1]
ltdate = dtxt(cprof.index[-1])['qtr2']
tot = cprof.loc[dt, ['A054RC', 'B056RC', 'A127RC']].sum()
totsh = tot / cprof.loc[dt, 'NNI'] * 100
div = cprof.loc[dt, 'B056RC']
divsh = div / cprof.loc[dt, 'NNI'] * 100
ret = cprof.loc[dt, 'A127RC']
tax = cprof.loc[dt, 'A054RC']

text = (f'In {ltdate}, aggregate corporate profits were \${tot:.2f} trillion, or {totsh:.1f} '+
        f'percent of net national income. Of this, \${div:.2f} trillion, equivalent to {divsh:.1f} '+
        'percent of net national product, were paid out as dividends (see\cbox{blue!70!purple}), '+
        f'\${ret*1000:.0f} billion were retained (see\cbox{{cyan!50!white}}), and \${tax*1000:.0f} '+
        'billion went to corporate income tax (see\cbox{red!80!orange}). ')
write_txt(text_dir / 'cprof.txt', text)
print(text)

In the third quarter of 2020, aggregate corporate profits were \$2.33 trillion, or 15.0 percent of net national income. Of this, \$1.35 trillion, equivalent to 8.7 percent of net national product, were paid out as dividends (see\cbox{blue!70!purple}), \$673 billion were retained (see\cbox{cyan!50!white}), and \$307 billion went to corporate income tax (see\cbox{red!80!orange}). 


### Corporate profits source

In [48]:
s = ['W170RC', 'A262RC', 'W986RC', 'A922RC']
df1 = nipa_df(retrieve_table('T50100')['Data'], s)

s = ['A123RC']
df2 = nipa_df(retrieve_table('T40100')['Data'], s)

s = ['A001RC']
df3 = nipa_df(retrieve_table('T10705')['Data'], s)

cprof = pd.DataFrame()
cprof['ROW Saving'] = (df2['A123RC'] / df3['A001RC']) * 100
cprof['HH Saving'] = (- df1['W986RC'] / df3['A001RC']) * 100
cprof['Gov Saving'] = (- df1['A922RC'] / df3['A001RC']) * 100
cprof['Investment'] = ((df1['W170RC'] - df1['A262RC']) / df3['A001RC']) * 100

cprof.loc['1989':].to_csv(data_dir / 'cprof2.csv', index_label='date')

In [48]:
#nipa_series_codes(retrieve_table('T50100'))

### Labor Productivity

In [6]:
# Series stored as a dictionary
series = {'PRS85006092': 'value',
          'PRS85006032': 'hours',
          'PRS85006042': 'output',
          'PRS84006033': 'hours_index'}

In [7]:
# Start year and end year
dates = (1989, 2021)
df = bls_api(series, dates, bls_key)

df.to_csv(data_dir / 'lprod.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [8]:
df = pd.read_csv(data_dir / 'lprod.csv')
df = df.set_index(pd.to_datetime(df['date']))
df = df.drop('date',axis=1)

d = series_info(df['value'])

s = {}

for srs in list(series.values()):
    s[srs] = {}
    tmp = series_info(df[srs])
    for i in ['val_latest', 'val_prev']:
        if tmp[i] > 0:
            s[srs][i] = f'increased at an annual rate of {tmp[i]:.1f} percent'
            s[srs][i+'2'] = f'an increase of {tmp[i]:.1f} percent'
        elif tmp[i] < 0:
            s[srs][i] = f'decreased at an annual rate of {abs(tmp[i]):.1f} percent'
            s[srs][i+'2'] = f'a decrease of {abs(tmp[i]):.1f} percent'
        else:
            s[srs][i] = 'was unchanged'
            s[srs][i+'2'] = 'no change'   
            
compare = compare_text(d["five_year_mean"], d["mean"], [0.1, 0.5, 2.0])
    
text = (f'In {d["date_latest_ft"]}, labor productivity {s["value"]["val_latest"]} '+
        f'(see\cbox{{teal}}), as the result of {s["output"]["val_latest2"]} in real ouput '+
        f'and {s["hours"]["val_latest2"]} in hours worked. In the prior quarter, '+
        f'{d["date_prev_ft"]}, labor productivity {s["value"]["val_prev"]}, as '+
        f'real output {s["output"]["val_prev"]} and hours of work {s["hours"]["val_prev"]}. '+
        f'Over the past five years, '+
        f'labor productivity growth has averaged {d["five_year_mean"]:.1f} percent, '+
        f'{compare} the 1989-onward average of {d["mean"]:.1f} percent.')

write_txt(text_dir / 'lprod.txt', text)
print(text)

text = (f'Total hours worked in nonfarm businesses {s["hours"]["val_latest"]} '+
        f'in {d["date_latest_ft"]}, following {s["hours"]["val_prev2"]} in '+
        f'{d["date_prev_ft"]}.')

write_txt(text_dir / 'tot_hours.txt', text)
print(text)

In 2020 Q4, labor productivity decreased at an annual rate of 4.2 percent (see\cbox{teal}), as the result of an increase of 5.5 percent in real ouput and an increase of 10.1 percent in hours worked. In the prior quarter, 2020 Q3, labor productivity increased at an annual rate of 4.2 percent, as real output increased at an annual rate of 44.1 percent and hours of work increased at an annual rate of 38.3 percent. Over the past five years, labor productivity growth has averaged 1.6 percent, slightly below the 1989-onward average of 2.0 percent.
Total hours worked in nonfarm businesses increased at an annual rate of 10.1 percent in 2020 Q4, following an increase of 38.3 percent in 2020 Q3.


### Gross Labor Income

In [52]:
s = {'LNS12005054': 'Hours', 'LNS12000000': 'Employment'}
df = bls_api(s, (1989, 2021), bls_key)
df['Total'] = df['Hours'] * df['Employment']
df.to_csv(data_dir / 'emp_hrs.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [53]:
emp = (pd.read_csv(data_dir / 'emp_hrs.csv', parse_dates=[0])
         .set_index('date')['Total'])
coe = nipa_df(retrieve_table('T20100')['Data'], ['A033RC'])
pce = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])
data = coe.join(pce).join(emp.resample('QS').mean()).dropna()
data['real_coe'] = data['A033RC'] / (data['DPCERG'] / data['DPCERG'].iloc[-1])
data['coe_inp'] = data['real_coe'] / data['Total']
data['wage'] = data['coe_inp'] * data['Total'].iloc[0]
data['work'] = data['real_coe'] - data['wage']

# Calculate contributions to growth
result = growth_contrib(data, 'real_coe')[['work', 'wage']]
result.dropna().loc['1989':]
result.to_csv(data_dir / 'gli.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
totval = result.iloc[-1].sum()
wage = result['wage'].iloc[-1]
work = result['work'].iloc[-1]

incdec = 'increased' if totval > 0 else 'decreased'
txt1 = (f'{incdec} at an annualized and inflation-adjusted '+
        f'rate of {abs(totval):.2f} percent')

if totval == 0:
    txt1 = 'was unchanged, after adjusting for inflation,'
    
txt2 = cont_subt(wage, style='end')
txt3 = cont_subt(work, style='end')

text = (f'{txt1} in {ltdate}. Changes in wages {txt2}, '+
        f'and changes in total hours worked {txt3}.')
write_txt(text_dir / 'gli.txt', text)
print(text)

increased at an annualized and inflation-adjusted rate of 7.01 percent in 2020 Q4. Changes in wages subtracted 2.08 percentage points, and changes in total hours worked contributed 9.09 percentage points.


### Financial Obligations Ratio

In [5]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=FOR&series=6d253f59629f788ab250a7a30662e79e&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2021&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d = {k: v.replace(', seasonally adjusted', '') for k, v in d.items()}

data = clean_data.rename(d, axis=1)['Financial obligations ratio']
data.to_csv(data_dir / 'for.csv', index_label='date')

node = end_node3(data, 'blue!80!black')
write_txt(text_dir / 'for_node.txt', node)

ltdate = dtxt(data.index[-1])['qtr1']
ltval = data.iloc[-1]

text = (f'As of {ltdate} the financial obligations ratio is {ltval:.1f} percent')
write_txt(text_dir / 'for.txt', text)
print(text)

As of 2020 Q3 the financial obligations ratio is 14.3 percent


### Shiller real return trailing 20-year average

In [58]:
data = pd.read_excel('http://www.econ.yale.edu/~shiller/data/ie_data.xls', sheet_name='Data')
df1 = data.iloc[679:1807, np.r_[0, 14]]
df1.columns = ['Date', 'CATRPE']
df1.index = pd.to_datetime([f'{val:.2f}' for val in df1['Date'].values])
df1 = df1.loc[~df1.index.duplicated(keep='first')]
df1.loc['1989':, 'CATRPE'].to_csv(data_dir / 'catrpe.csv', index_label='date')

node = end_node(df1.CATRPE, 'blue!80!black', percent=False, date=True)
write_txt(text_dir / 'cape_node.txt', node)

df = data.iloc[679:1807, np.r_[0, 7, 8]]
df.columns = ['Date', 'Price', 'Dividend']
df.index = pd.to_datetime([f'{val:.2f}' for val in df['Date'].values])
df = df.loc[~df.index.duplicated(keep='first')]

In [59]:
for yrs in [10, 15, 20]:
    s = df.copy()
    mos = yrs * 12
    s['DY'] = (s['Dividend'] / s['Price']).rolling(mos).mean()
    s['Pch'] = s['Price'].pct_change(mos)
    s['Pch'] = (s['Pch']+1)**(1/yrs) - 1
    s = s.dropna()
    s['Return'] = (s['DY'] + s['Pch']) * 100
    s.loc['1989':].dropna().to_csv(data_dir / f'sp500rr{yrs}.csv', 
                                   index_label='date', float_format='%g')
    
text = ('According to historical stock market '+
        '\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, '+
        'the inflation-adjusted trailing twenty year annual rate of return '+
        f'of the S\&P 500 was {s["Return"].iloc[-1]:.1f} percent as '+
        f'of {s.index[-1].strftime("%B %Y")}. Real returns are currently low relative '+
        'to the average trailing twenty year real annual return of '+
        f'{s["Return"].loc["1995":"2005"].mean():.1f} percent during 1995--2005.')

write_txt(text_dir / 'sp500rr3.txt', text)

print(text)

According to historical stock market \href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the inflation-adjusted trailing twenty year annual rate of return of the S\&P 500 was 5.1 percent as of December 2020. Real returns are currently low relative to the average trailing twenty year real annual return of 10.1 percent during 1995--2005.


### High Quality Corporate Bond Yield

In [4]:
df = fred_df('HQMCB10YR')
df.to_csv(data_dir / 'hqcb.csv', index_label='date')
node = end_node(df['VALUE'], 'blue!60!violet', date=True)
write_txt(text_dir / 'hqcb_node.txt', node)

ltval = df.VALUE.iloc[-1]
prval = df.VALUE.iloc[-2]
prval2 = df.VALUE.iloc[-13]
ltdate = dtxt(df.index[-1])['mon1']
prdate = dtxt(df.index[-2])['mon1']
prdate2 = dtxt(df.index[-13])['mon1']

text = (f'This monthly measure shows a spot rate of {ltval:.1f} '+
        f'percent in {ltdate} for high-quality corporate bonds with '+
        f'a maturity of 10 years, following a rate of {prval:.1f} percent '+
        f'in {prdate}. In {prdate2}, the spot rate was {prval2:.1f} percent.')

write_txt(text_dir / 'hqcb.txt', text)
print(text)

This monthly measure shows a spot rate of 2.2 percent in January 2021 for high-quality corporate bonds with a maturity of 10 years, following a rate of 2.1 percent in December 2020. In January 2020, the spot rate was 2.8 percent.


### International Investment Position (IIP)

In [3]:
series = ['IIPUSLIAA', 'IIPUSASSA','GDPA']
df = pd.DataFrame()
for s in series:
    df[s] = fred_df(s)['VALUE']
    
series2 = ['IIPUSLIAQ', 'IIPUSASSQ', 'GDP']
df2 = pd.DataFrame()
for s in series2:
    df2[s] = fred_df(s)['VALUE']
    
data = pd.DataFrame()
data['Liab'] = (df.loc[:'2005', 'IIPUSLIAA'] / df.loc[:'2005', 'GDPA']) / 10
data['Assets'] = (df.loc[:'2005', 'IIPUSASSA'] / df.loc[:'2005', 'GDPA']) / 10
data2 = pd.DataFrame()
data2['Liab'] = (df2.loc[:, 'IIPUSLIAQ'] / df2.loc[:, 'GDP']) / 10
data2['Assets'] = (df2.loc[:, 'IIPUSASSQ'] / df2.loc[:, 'GDP']) / 10
result = data.append(data2)
result.to_csv(data_dir / 'iip.csv', index_label='date')

lt = result.index[-1]
ltdt = dtxt(lt)['qtr1']
pr = '2006-01-01'
prdt = dtxt(pd.to_datetime(pr))['qtr1']
ltvala = result.loc[lt, 'Assets']
ltvall = result.loc[lt, 'Liab']
ltvaln = ltvala - ltvall
prvala = result.loc[pr, 'Assets']
prvall = result.loc[pr, 'Liab']
prvaln = prvala - prvall

text = (f'In {ltdt}, domestic holdings of foreign assets are equivalent '+
        f'to {ltvala:.1f} percent of GDP '+
        '(see {\color{blue!95!violet}\\textbf{---}}), compared to '+
        f'{prvala:.1f} percent in {prdt}. Domestic liabilities '+
        f'to the foreign sector total {ltvall:.1f} percent of GDP in {ltdt} '+
        '(see {\color{red}\\textbf{---}}), and '+
        f'{prvall:.1f} percent in {prdt}. As a result, net IIP, '+
        'holdings of foreign assets minus liabilities, identifies the '+
        'US as a net debtor to the result of the world, to the equivalent '
        f'of {abs(ltvaln):.1f} percent of GDP in {ltdt} '+
        '(see\cbox{cyan!25!white}) '+
        f'and {abs(prvaln):.1f} percent of GDP in {prdt}.')

write_txt(text_dir / 'niip.txt', text)
print(text)

In 2020 Q3, domestic holdings of foreign assets are equivalent to 138.9 percent of GDP (see {\color{blue!95!violet}\textbf{---}}), compared to 106.3 percent in 2006 Q1. Domestic liabilities to the foreign sector total 204.8 percent of GDP in 2020 Q3 (see {\color{red}\textbf{---}}), and 118.5 percent in 2006 Q1. As a result, net IIP, holdings of foreign assets minus liabilities, identifies the US as a net debtor to the result of the world, to the equivalent of 65.9 percent of GDP in 2020 Q3 (see\cbox{cyan!25!white}) and 12.2 percent of GDP in 2006 Q1.


### Consumer Credit (G.19)

The same chart also includes a line from the Z.1 notebook

In [2]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G19&series=8c7511a37e4aea678be81e7a61df57db&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2021&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d = {'DTCTL_N.M': 'level, nsa',
     'DTCTL_XDF_BA_N.M': 'flow, nsa',
     'DTCTL.M': 'level',
     'DTCTL_XDF_BA.M': 'flow'}

data = clean_data.rename(d, axis=1)[['level', 'flow']]
data['DPI'] = fred_df('DSPI')['VALUE']

rate = ((data['level'] / (data['DPI'] * 1000)) * 100)
rate.name = 'value'
rate.to_csv(data_dir / 'cc_dpi_monthly.csv', index_label='date')

node_color = 'green!70!blue!80!black'

node = end_node(rate, node_color, date=True)
write_txt(text_dir / 'cc_dpi_monthly_node.txt', node)

ltdate = dtxt(data.index[-1])['mon1']
ltval = data['level'].iloc[-1] / 1_000_000
one_yr = inc_dec_percent(data['level'].pct_change(12).iloc[-1] * 100)
one_yr_dpi = inc_dec_percent(data['DPI'].pct_change(12).iloc[-1] * 100)
one_yr_rate = val_inc_pp(rate.diff(12).iloc[-1])

ltrate = rate.iloc[-1]

text = (f'Consumer credit totals \${ltval:.2f} trillion US dollars on '+
        f'a seasonally-adjusted and annualized basis in {ltdate}. '+
        f'Over the past year, consumer credit {one_yr}, while '+
        f'after-tax income {one_yr_dpi}. As a result, the ratio '+
        f'of consumer credit to disposable income {one_yr_rate}. '+
        f'In {ltdate}, total consumer credit is equivalent to '+
        f'{ltrate:.1f} percent of annualized {ltdate} disposable '+
        f'income (see {{\color{{{node_color}}}\\textbf{{---}}}}). ')
print(text)
write_txt(text_dir / 'cc_dpi.txt', text)

Consumer credit totals \$4.18 trillion US dollars on a seasonally-adjusted and annualized basis in January 2021. Over the past year, consumer credit decreased by 0.2 percent, while after-tax income increased by 15.0 percent. As a result, the ratio of consumer credit to disposable income decreased by a total of 3.3 percentage points. In January 2021, total consumer credit is equivalent to 21.7 percent of annualized January 2021 disposable income (see {\color{green!70!blue!80!black}\textbf{---}}). 


### Employment Cost Index

In [63]:
series = {'CIU2020000000000A': 'PrivateWS',
          'CIU2030000000000A': 'PrivateBe'}

dates = (2001, 2020)
df = bls_api(series, dates, bls_key)

df.to_csv(data_dir/ 'eci.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [64]:
write_txt(text_dir/ 'eci_node.txt', end_node3(df['PrivateWS'], 'green!80!blue'))
write_txt(text_dir/ 'eci_node2.txt', end_node3(df['PrivateBe'], 'cyan!40!blue'))
ltdate = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
ltval = inc_dec_percent(df['PrivateWS'].iloc[-1])
ltbval = inc_dec_percent(df['PrivateBe'].iloc[-1])
prdate = dtxt(df.index[-2])['qtr1']
prval = inc_dec_percent(df['PrivateWS'].iloc[-2], how='of')
prbval = inc_dec_percent(df['PrivateBe'].iloc[-2], how='of')
prdate2 = dtxt(df.index[-3])['qtr1']
prval2 = inc_dec_percent(df['PrivateWS'].iloc[-3], how='of')

text = (f'In {ltdate}, private industry wage and salary costs '+
        f'(see {{\color{{green!80!blue}}\\textbf{{---}}}}) {ltval} '+
        '(12-month percent changes shown), '+
        f'following {prval} in {prdate}, and {prval2} in {prdate2}. '+
        f'Private sector benefit costs '+
        f'{ltbval} (see {{\color{{cyan!40!blue}}\\textbf{{---}}}}) '+
        f'over the 12-months ending {ltdt2}, following {prbval} in {prdate}.')

write_txt(text_dir / 'eci.txt', text)
print(text)

In the fourth quarter of 2020, private industry wage and salary costs (see {\color{green!80!blue}\textbf{---}}) increased by 2.8 percent (12-month percent changes shown), following an increase of 2.7 percent in 2020 Q3, and an increase of 2.9 percent in 2020 Q2. Private sector benefit costs increased by 2.1 percent (see {\color{cyan!40!blue}\textbf{---}}) over the 12-months ending 2020 Q4, following an increase of 2.0 percent in 2020 Q3.
